#  #Validation of Flap model

In [3]:
workspace()
include("../src/UNSflow.jl")

UNSflow

In [4]:
using UNSflow


# # Laura's case C - flap only

In [5]:
#Kinematics
alpha_amp = 0.
alpha_mean = 0.
alpha_zl = 0. #Flat plate
h_amp = 0. 
k = 3.93
beta_amp = 5*pi/180.

0.08726646259971647

In [6]:
#for UNSflow

w = 2*k 
T = (2*pi/w)
ncyc = 4
t_tot = ncyc*T 

dt = 0.015*0.2/(k*beta_amp) 
nsteps =round(Int,t_tot/dt)+1

alphadef = ConstDef(alpha_amp)
hdef = ConstDef(h_amp)
udef = ConstDef(1.)
ndef = CosDef(0., beta_amp, w, 0.)

full_kinem = KinemDefwFlap(alphadef, hdef, udef, ndef)

pvt = 0.2 #Doesnt matter for flap only

lespcrit = [21;] #high value to turn off LEV shedding

x_b = [0.75;]

surf = TwoDSurfwFlap(1., 1., "FlatPlate", pvt, 70, 35, "Prescribed", full_kinem,x_b, lespcrit)

curfield = TwoDFlowField()

UNSflow.TwoDFlowField(UNSflow.ConstDef(0.0),UNSflow.ConstDef(0.0),[0.0],[0.0],UNSflow.TwoDVort[],UNSflow.TwoDVort[])

In [7]:
ldvm(surf, curfield, nsteps, dt)


(UNSflow.TwoDSurfwFlap(1.0,1.0,"FlatPlate",0.2,70,35,"Prescribed",UNSflow.KinemDefwFlap(UNSflow.ConstDef(0.0),UNSflow.ConstDef(0.0),UNSflow.ConstDef(1.0),UNSflow.CosDef(0.0,0.08726646259971647,7.86,0.0)),[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  -0.0181584,-0.0189062,-0.0195698,-0.0201477,-0.0206387,-0.0210418,-0.0213563,-0.0215813,-0.0217165,-0.0217616],[-2.56758e-28,1.92535e-28,-7.69743e-28,3.7813e-27,-1.70529e-26,7.29865e-26,-3.02997e-25,1.23329e-24,-4.9509e-24,1.96715e-23  …  -0.0870466,-0.0870466,-0.0870466,-0.0870466,-0.0870466,-0.0870466,-0.0870466,-0.0870466,-0.0870466,-0.0870466],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0144389,0.0150335,0.0155612,0.0160207,0.0164111,0.0167317,0.0169817,0.0171606,0.0172682,0.017304],[0.0,0.0455303,0.0910607,0.136591,0.182121,0.227652,0.273182,0.318712,0.364243,0.409773  …  2.73182,2.77735,2.82288,2.86841,2.91394,2.95947,3.005,3.05053,3.09606,3.14159]

In [8]:
#for Theodorsen
theo_in = TheoDefwFlap(alpha_amp, h_amp, alpha_mean, alpha_zl, k, 0., pvt, beta_amp, x_b[1], 0.) 

UNSflow.TheoDefwFlap(0.0,0.0,0.0,0.0,3.93,0.0,0.2,0.08726646259971647,0.75,0.0)

In [9]:
(t_theo, cl, cm_alpha, cm_beta) = theodorsen(theo_in)

([0.0,0.00277778,0.00555556,0.00833333,0.0111111,0.0138889,0.0166667,0.0194444,0.0222222,0.025  …  0.975,0.977778,0.980556,0.983333,0.986111,0.988889,0.991667,0.994444,0.997222,1.0],Complex{Float64}[0.012327+0.424731im,0.0049125+0.424881im,-0.00250345+0.424902im,-0.00991863+0.424794im,-0.0173308+0.424556im,-0.0247377+0.424189im,-0.032137+0.423693im,-0.0395266+0.423067im,-0.0469041+0.422313im,-0.0542674+0.42143im  …  0.0786177+0.417573im,0.0713181+0.418882im,0.0639968+0.420063im,0.0566559+0.421116im,0.0492978+0.42204im,0.0419247+0.422836im,0.0345388+0.423504im,0.0271423+0.424042im,0.0197377+0.424451im,0.012327+0.424731im],Complex{Float64}[0.0250906-0.233046im,0.029154-0.232572im,0.0332085-0.232028im,0.0372529-0.231413im,0.0412859-0.230728im,0.0453064-0.229972im,0.0493131-0.229147im,0.0533047-0.228251im,0.0572801-0.227286im,0.0612381-0.226252im  …  -0.0116747-0.234102im,-0.0075873-0.23427im,-0.00349757-0.234367im,0.000593221-0.234392im,0.00468383-0.234346im,0.00877302-0.234228im,0.012859

In [10]:
#Plots to compare

#UNSflow
data = readdlm("results.dat")
range = round(Int,(ncyc-1)*nsteps/ncyc)+1:nsteps
tbyT = (data[range,1]-data[range[1]])/T

plot(tbyT, data[range,6])
plot(t_theo, imag(cl))


1-element Array{Any,1}:
 PyObject <matplotlib.lines.Line2D object at 0x7f11ba1d3ad0>

# It seems to be working but for some reason, the imaginary part of Theodorsen is giving the correct value. It should be the real part for a cosine function. Possibly a missing i.

In [9]:
data[range,9]

92-element Array{Float64,1}:
 -4.47688
 -4.29707
 -4.09619
 -3.87522
 -3.63523
 -3.37737
 -3.10287
 -2.81305
 -2.50928
 -2.193  
 -1.86569
 -1.52889
 -1.18417
  ⋮      
 -4.91238
 -5.00666
 -5.07663
 -5.12194
 -5.14237
 -5.13784
 -5.10836
 -5.05408
 -4.97529
 -4.87237
 -4.74583
 -4.59631

# Lets try a sine flap

In [10]:
ndef = SinDef(0., beta_amp, w, 0.)
# everything else is unchanged
full_kinem = KinemDefwFlap(alphadef, hdef, udef, ndef)


UNSflow.KinemDefwFlap(UNSflow.ConstDef(0.0),UNSflow.ConstDef(0.0),UNSflow.ConstDef(1.0),UNSflow.SinDef(0.0,0.08726646259971647,7.86,0.0))

In [11]:
surf = TwoDSurfwFlap(1., 1., "FlatPlate", pvt, 70, 35, "Prescribed", full_kinem,x_b, lespcrit)

curfield = TwoDFlowField()

UNSflow.TwoDFlowField(UNSflow.ConstDef(0.0),UNSflow.ConstDef(0.0),[0.0],[0.0],UNSflow.TwoDVort[],UNSflow.TwoDVort[])

In [12]:
ldvm(surf, curfield, nsteps, dt)

(UNSflow.TwoDSurfwFlap(1.0,1.0,"FlatPlate",0.2,70,35,"Prescribed",UNSflow.KinemDefwFlap(UNSflow.ConstDef(0.0),UNSflow.ConstDef(0.0),UNSflow.ConstDef(1.0),UNSflow.SinDef(0.0,0.08726646259971647,7.86,0.0)),[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  -0.00182324,-0.00189833,-0.00196495,-0.00202298,-0.00207228,-0.00211276,-0.00214433,-0.00216692,-0.0021805,-0.00218503],[-2.57804e-29,1.9332e-29,-7.72879e-29,3.79671e-28,-1.71224e-27,7.32839e-27,-3.04231e-26,1.23832e-25,-4.97107e-25,1.97516e-24  …  -0.00874012,-0.00874012,-0.00874012,-0.00874012,-0.00874012,-0.00874012,-0.00874012,-0.00874012,-0.00874012,-0.00874012],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  -0.142377,-0.148241,-0.153444,-0.157975,-0.161825,-0.164986,-0.167451,-0.169216,-0.170276,-0.170629],[0.0,0.0455303,0.0910607,0.136591,0.182121,0.227652,0.273182,0.318712,0.364243,0.409773  …  2.73182,2.77735,2.82288,2.86841,2.91394,2.95947,3.005,3.0

In [13]:
#Plots to compare

#UNSflow
data = readdlm("results.dat")
range = round(Int,(ncyc-1)*nsteps/ncyc)+1:nsteps
tbyT = (data[range,1]-data[range[1]])/T

plot(tbyT, data[range,6])
plot(t_theo, -real(cl_theo))

LoadError: LoadError: UndefVarError: cl_theo not defined
while loading In[13], in expression starting on line 9

# Negative of the real value corresponds to the sine results. So it seems like a extra i, not a missing one. In any case, the LDVM implementation seems right 

# # Variation of Laura's case E - Pitch and flap

In [14]:
#Kinematics
alpha_amp = 5*pi/180
alpha_mean = 0.
alpha_zl = 0. #Flat plate
h_amp = 0. 
k = 3.93
beta_amp = 5*pi/180.

0.08726646259971647

In [15]:
#for UNSflow

w = 2*k 
T = (2*pi/w)
ncyc = 4
t_tot = ncyc*T 

dt = 0.015*0.2/(k*alpha_amp) 
nsteps =round(Int,t_tot/dt)+1

367

In [16]:
#for UNSflow

w = 2*k 
T = (2*pi/w)
ncyc = 4
t_tot = ncyc*T 

dt = 0.015*0.2/(k*beta_amp) 
nsteps =round(Int,t_tot/dt)+1

alphadef = CosDef(alpha_mean, alpha_amp, w, 0.)
hdef = ConstDef(h_amp)
udef = ConstDef(1.)
ndef = CosDef(0., beta_amp, w, 0.)

full_kinem = KinemDefwFlap(alphadef, hdef, udef, ndef)

pvt = 0.2 

lespcrit = [21;] #high value to turn off LEV shedding

x_b = [0.75;]

surf = TwoDSurfwFlap(1., 1., "FlatPlate", pvt, 70, 35, "Prescribed", full_kinem,x_b, lespcrit)

curfield = TwoDFlowField()

UNSflow.TwoDFlowField(UNSflow.ConstDef(0.0),UNSflow.ConstDef(0.0),[0.0],[0.0],UNSflow.TwoDVort[],UNSflow.TwoDVort[])

In [17]:
ldvm(surf, curfield, nsteps, dt)

(UNSflow.TwoDSurfwFlap(1.0,1.0,"FlatPlate",0.2,70,35,"Prescribed",UNSflow.KinemDefwFlap(UNSflow.CosDef(0.0,0.08726646259971647,7.86,0.0),UNSflow.ConstDef(0.0),UNSflow.ConstDef(1.0),UNSflow.CosDef(0.0,0.08726646259971647,7.86,0.0)),[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  -0.0181584,-0.0189062,-0.0195698,-0.0201477,-0.0206387,-0.0210418,-0.0213563,-0.0215813,-0.0217165,-0.0217616],[-2.56758e-28,1.92535e-28,-7.69743e-28,3.7813e-27,-1.70529e-26,7.29865e-26,-3.02997e-25,1.23329e-24,-4.9509e-24,1.96715e-23  …  -0.0870466,-0.0870466,-0.0870466,-0.0870466,-0.0870466,-0.0870466,-0.0870466,-0.0870466,-0.0870466,-0.0870466],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0144389,0.0150335,0.0155612,0.0160207,0.0164111,0.0167317,0.0169817,0.0171606,0.0172682,0.017304],[0.0,0.0455303,0.0910607,0.136591,0.182121,0.227652,0.273182,0.318712,0.364243,0.409773  …  2.73182,2.77735,2.82288,2.86841,2.91394,2.95947,3.0

In [18]:
#for Theodorsen
theo_in = TheoDefwFlap(alpha_amp, h_amp, alpha_mean, alpha_zl, k, 0., pvt, beta_amp, x_b[1], 0.)
(t_theo, _, cl_h, cl_alpha, cl_beta, cl_theo) = theodorsen(theo_in)

LoadError: LoadError: BoundsError: attempt to access ([0.0,0.00277778,0.00555556,0.00833333,0.0111111,0.0138889,0.0166667,0.0194444,0.0222222,0.025  …  0.975,0.977778,0.980556,0.983333,0.986111,0.988889,0.991667,0.994444,0.997222,1.0],Complex{Float64}[-2.17851+2.67933im,-2.22493+2.6409im,-2.27069+2.60167im,-2.31574+2.56164im,-2.3601+2.52084im,-2.40373+2.47926im,-2.44664+2.43694im,-2.48879+2.39386im,-2.53019+2.35006im,-2.57082+2.30555im  …  -1.73254+2.98714im,-1.78441+2.95644im,-1.83574+2.92485im,-1.8865+2.89237im,-1.9367+2.859im,-1.9863+2.82477im,-2.03529+2.78967im,-2.08367+2.75373im,-2.13141+2.71694im,-2.17851+2.67933im],Complex{Float64}[1.03943-0.803071im,1.05329-0.784808im,1.06683-0.766306im,1.08004-0.74757im,1.09292-0.728607im,1.10547-0.709422im,1.11768-0.690021im,1.12956-0.67041im,1.14109-0.650594im,1.15227-0.63058im  …  0.90101-0.955787im,0.917553-0.939917im,0.933817-0.92376im,0.949797-0.907322im,0.965487-0.890608im,0.980883-0.873622im,0.995981-0.85637im,1.01077-0.838857im,1.0252

In [19]:
#Plots to compare

#UNSflow
data = readdlm("results.dat")
range = round(Int,(ncyc-1)*nsteps/ncyc)+1:nsteps
tbyT = (data[range,1]-data[range[1]])/T

plot(tbyT, data[range,6])
plot(t_theo, real(cl_theo))


LoadError: LoadError: UndefVarError: cl_theo not defined
while loading In[19], in expression starting on line 9

# Looks good. alpha=10 + flap=5 becomes large angle and we start seeing differences between LDVM and theodorsen